In [50]:
import numpy as np
import random
from sklearn import datasets

In [51]:
dim = 4
#aktivasyon fonksiyonu ve türevi
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def der_sigmoid(x):
    return np.exp(x)/((np.exp(x)+1)**2)

#import data
f = open("iris.data", "r")
data = f.read()
data_list = data.split("\n")

seperated = []
virginica = []
versicolor = []
setosa = []
#labelling classes
for i in range(len(data_list)):
    seperated = data_list[i].split(",")
    sl = float(seperated[0])
    sw = float(seperated[1])
    pl = float(seperated[2])
    pw = float(seperated[3])

    if seperated[4] == "Iris-virginica":
        virginica.append([[sl, sw, pl, pw], [0, 0, 1]])
    elif seperated[4] == "Iris-versicolor":
        versicolor.append([[sl, sw, pl, pw], [0, 1, 0]])
    elif seperated[4] == "Iris-setosa":
        setosa.append([[sl, sw, pl, pw], [1, 0, 0]])
#shuffle the data
random.shuffle(virginica)
random.shuffle(versicolor)
random.shuffle(setosa)
#generating test and training set
train_data = np.concatenate((setosa[:40], np.concatenate((virginica[:40], setosa[:40]))))
test_data = np.concatenate((setosa[40:], np.concatenate((virginica[40:], setosa[40:]))))

#gizli katman sayısını ve nöron sayısını belirtelim
input_dimension = 4
gk_1 = 5
gk_2 = 4
out = 3
#başlangıç ağırlıkları
weights_1 = (np.random.rand(gk_1, input_dimension + 1)-0.5)/1
weights_2 = (np.random.rand(gk_2, gk_1 + 1)-0.5)/1
weights_out = (np.random.rand(out, gk_2 + 1)-0.5)/1

#bir önceki ağırlıklar
weights_1_old = 0
weights_2_old = 0
weights_out_old = 0
Error = 0
n_learn = 0.01  #öğrenme katsayısı
alpha = 0.9     #momentum terim katsayısı

#training the network
for j in range(1000):
    Error = 0
    for i in range(len(train_data)):
        step_data = train_data[i][0]
        step_data = np.append(step_data, [1])

        vgk1_step = np.matmul(weights_1, step_data)
        y1 = sigmoid(vgk1_step)
        y1 = np.append(y1, 1)

        vgk2_step = np.matmul(weights_2, y1)
        y2 = sigmoid(vgk2_step)
        y2 = np.append(y2, 1)

        vo_step = np.matmul(weights_out, y2)
        yo = sigmoid(vo_step)

        step_error = yo - train_data[i][1]
        #yerel gradyenler
        grd_o = step_error * der_sigmoid(vo_step)
        grd_2 = np.matmul(np.transpose(weights_out)[:-1], grd_o) * der_sigmoid(vgk2_step)
        grd_1 = np.matmul(np.transpose(weights_2)[:-1], grd_2) * der_sigmoid(vgk1_step)
        #ağırlık güncellenmesi
        weights_1_new = weights_1 - n_learn * np.matmul(grd_1.reshape(-1, 1), np.transpose(step_data.reshape(-1, 1))) + alpha * (weights_1 - weights_1_old)
        weights_2_new = weights_2 - n_learn * np.matmul(grd_2.reshape(-1, 1), np.transpose(y1.reshape(-1, 1))) + alpha * (weights_2 - weights_2_old)
        weights_out_new = weights_out - n_learn * np.matmul(grd_o.reshape(-1, 1), np.transpose(y2.reshape(-1, 1))) + alpha * (weights_out - weights_out_old)
        #bir önceki ağırlıkları kaydedelim
        weights_1_old = weights_1
        weights_2_old = weights_2
        weights_out_old = weights_out
        #ağırlıkların güncellenmesini sağlayalım
        weights_1 = weights_1_new
        weights_2 = weights_2_new
        weights_out = weights_out_new

        Error = Error + 0.5*np.matmul(step_error, np.transpose(step_error))

    if Error/len(train_data) < 0.0001:  #durdurma kriteri
        print(j)
        break

print(Error/len(train_data))

#testing section
Error = 0
prediction = []
for i in range(len(test_data)):
    step_data = test_data[i][0]
    step_data = np.append(step_data, [1])

    vgk1_step = np.matmul(weights_1, step_data)
    y1 = sigmoid(vgk1_step)
    y1 = np.append(y1, 1)

    vgk2_step = np.matmul(weights_2, y1)
    y2 = sigmoid(vgk2_step)
    y2 = np.append(y2, 1)

    vo_step = np.matmul(weights_out, y2)
    yo = sigmoid(vo_step)

    step_error = yo - test_data[i][1]

    Error = Error + 0.5*np.matmul(step_error, np.transpose(step_error))
    yo = np.where(yo<0.5,0,1)
print(Error/len(test_data))

643
9.992996880498332e-05
9.784499525841814e-05


In [52]:
1-Error/len(test_data)

0.9999021550047416